<a href="https://www.kaggle.com/code/subram/formality-style-transfer-finetuned?scriptVersionId=208552232" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [4]:
file_path = '/kaggle/input/gyafcdatae/Entertainment_Music/train/informal'   # Replace 'your_text_file.txt' with the path to your text file
with open(file_path, 'r') as file:
    X_train = file.readlines()

for i  in range(len(X_train)):
  X_train[i] = X_train[i].split("\n")[0]
  X_train[i] = X_train[i].lower()

In [ ]:
print(len(X_train))
X_train[30000:30010]

In [5]:
# Read text from a file
file_path = '/kaggle/input/gyafcdatae/Entertainment_Music/train/formal'
with open(file_path, 'r') as file:
    y_train = file.readlines()

for i  in range(len(y_train)):
  y_train[i] = y_train[i].split("\n")[0]
  y_train[i] = y_train[i].lower()

In [ ]:
print(len(y_train))
y_train[30000:30010]

In [6]:
# Read text from a file
file_path = '/kaggle/input/gyafcdatae/Entertainment_Music/test/informal'
with open(file_path, 'r') as file:
    X_test = file.readlines()

for i  in range(len(X_test)):
  X_test[i] = X_test[i].split("\n")[0]
  X_test[i] = X_test[i].lower()

print(len(X_test))
X_test[:10]

1416


['is any baby really a freak.',
 'aspen colorado has he best music festivals, you sit all over the moutians its  on and just hang out',
 'you can get almost anything on ebay!',
 'everybody is dying to get in',
 'not idiots like 50 cent and his whole gay unit.those kinds of ppl give hip-hop a bad name.',
 "different from what i've seen though",
 'i want to be on tv!',
 'dont let anyone decide the fate but you.',
 '50 is just riding coattails with that movie.',
 'the blind klan guy is hilarious!']

In [7]:
# Read text from a file
file_path = '/kaggle/input/gyafcdatae/Entertainment_Music/test/formal.ref2'
with open(file_path, 'r') as file:
    y_test = file.readlines()

for i  in range(len(y_test)):
  y_test[i] = y_test[i].split("\n")[0]
  y_test[i] = y_test[i].lower()

print(len(y_test))
y_test[:10]


1416


['is any baby really strange?',
 'aspen colorado is the home to some great music festivals, you can sit back enjoy the mountains and the music.',
 'you can purchase almost anything you want on ebay!',
 'everyone really wants to get in there/',
 'not idiots like 50 cent and his whole deplorable unit. those kinds of people give hip-hop music a bad name.',
 'differently from what i have seen, however.',
 'i want to appear on television!',
 "don't let anyone else decide your fate.",
 "i think he is just riding others' coattails with that movie.",
 'the blind klansman is very funny.']

In [ ]:
from torch.utils.data import Dataset
import torch

class ChatData(Dataset):
    def __init__(self, tokenizer, X_train, y_train, subset_range=(0, None), max_length=64):
        """
        Initializes the dataset for fine-tuning GPT-2.
        
        Args:
        - tokenizer: The tokenizer to use for encoding data.
        - X_train: List of informal sentences.
        - y_train: List of corresponding formal sentences.
        - subset_range: Tuple specifying the range of data to use (start_idx, end_idx).
        - max_length: Maximum length for tokenization (default=64).
        """
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.X = X_train
        self.y = y_train
        
        # Apply subset range if specified
        start, end = subset_range
        end = end if end is not None else len(self.X)
        self.X = self.X[start:end]
        self.y = self.y[start:end]
        
        # Format the data with prefixes and suffixes
        formatted_data = []
        for idx, (i, target) in enumerate(zip(self.X, self.y)):
            try:
                formatted_data.append(
                    f"<|startoftext|>\n[Informal]: {i}\n [Formal]: {target} <|endoftext|>"
                )
            except Exception as e:
                print(f"Error processing index {idx}: {e}")
        
        self.X = formatted_data
        print(f"Sample input after formatting: {self.X[0]}")  # Debugging: print first example

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        """
        Tokenizes the sample on-the-fly to avoid memory issues.
        """
        text = self.X[idx]
        encoded = self.tokenizer(
            text, 
            max_length=self.max_length, 
            truncation=True, 
            padding="max_length", 
            return_tensors="pt"
        )
        input_ids = encoded['input_ids'].squeeze(0)
        attention_mask = encoded['attention_mask'].squeeze(0)
        return input_ids, attention_mask


In [ ]:
!pip install --upgrade jupyterlab jupyterlab_widgets

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch

# Training function
def train(chatData, model, tokenizer, optim, device):
    epochs = 20
    for epoch in tqdm.tqdm(range(epochs), desc="Training"):
        model.train()  # Ensure model is in training mode
        total_loss = 0
        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)
            
            optim.zero_grad()
            outputs = model(X, attention_mask=a, labels=X)
            loss = outputs.loss
            loss.backward()
            optim.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(chatData):.4f}")

        # Save tokenizer and model configurations
        tokenizer.save_pretrained("tokenizer_configs")
        model.save_pretrained("model_configs")
        # torch.save(model.state_dict(), f"model_state_epoch_{epoch + 1}.pt")
        torch.save(model.state_dict(), "model_state.pt")
        
        # Run inference for debugging
        print("Example output:", infer('Damn,u look fine!', tokenizer, model, device))

# Inference function
def infer(inp, tokenizer, model, device):
    inp = f"<|startoftext|>\n[Informal]: {inp}\n [Formal]: "
    encoded = tokenizer(
        inp, 
        return_tensors="pt", 
        max_length=30, 
        truncation=True, 
        padding="max_length"
    )
    X = encoded["input_ids"].to(device)
    a = encoded["attention_mask"].to(device)
    
    output = model.generate(
        X, attention_mask=a, max_new_tokens=10)
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return output_text

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenizer setup
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
tokenizer.add_special_tokens({
    "pad_token": "<pad>",
    "bos_token": "<|startoftext|>",
    "eos_token": "<|endoftext|>"
})
tokenizer.add_tokens(["[Informal]:", "[Formal]:"])

# Model setup
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
model.resize_token_embeddings(len(tokenizer))  # Adjust for new tokens
model = model.to(device)

# Dataset and DataLoader
chatData = ChatData(tokenizer, X_train, y_train, subset_range=(0, len(X_train)), max_length=30)
chatDataLoader = DataLoader(chatData, batch_size=16, shuffle=True)

# Optimizer
optim = Adam(model.parameters(), lr=1e-4)

# Training
print("Starting training...")
train(chatDataLoader, model, tokenizer, optim, device)

In [2]:
import os
import shutil

# Ensure the destination directory exists
kaggle_config_dir = os.path.expanduser('~/.config/kaggle/')
os.makedirs(kaggle_config_dir, exist_ok=True)

# Move the kaggle.json file
shutil.copy('/kaggle/input/kaggles/kaggle.json', os.path.join(kaggle_config_dir, 'kaggle.json'))

print("kaggle.json file moved successfully!")


kaggle.json file moved successfully!


In [ ]:
!kaggle kernels output subram/notebook6583ea48bc -p /kaggle/working

In [8]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("/kaggle/working/tokenizer_configs")
model = GPT2LMHeadModel.from_pretrained("/kaggle/working/model_configs")

In [ ]:
!pip install sentence_transformers

In [9]:
import torch

def infer(inp, tokenizer, model, device):
    inp = f"<|startoftext|>\n[Informal]: {inp}\n [Formal]: "
    encoded = tokenizer(
        inp, 
        return_tensors="pt", 
        max_length=30, 
        truncation=True, 
        padding="max_length"
    )
    X = encoded["input_ids"].to(device)
    a = encoded["attention_mask"].to(device)
    
    output = model.generate(
        X, attention_mask=a, max_new_tokens=10)
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return output_text

# Ensure proper device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move model to the correct device


In [ ]:
inp = input()
print(infer(inp, tokenizer, model, device))
# print(infer(inp))

In [3]:
!kaggle kernels output subram/evaluation -p /kaggle/working

Output file downloaded to /kaggle/working/model_configs_fscore/config.json
Output file downloaded to /kaggle/working/model_configs_fscore/model.safetensors
Output file downloaded to /kaggle/working/tokenizer_configs_fscore/special_tokens_map.json
Output file downloaded to /kaggle/working/tokenizer_configs_fscore/tokenizer_config.json
Output file downloaded to /kaggle/working/tokenizer_configs_fscore/vocab.txt
Kernel log downloaded to /kaggle/working/evaluation.log 


In [20]:
generated_outputs = []

for i in range(len(X_test)):
    output = infer(X_test[i+5], tokenizer, model, device)
    formal_part = output.split("[Formal]:")[-1].strip()
    generated_outputs.append(formal_part)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin

IndexError: list index out of range

In [38]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

# Step 1: Load the Universal Sentence Encoder
print("Loading Universal Sentence Encoder...")
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
print("USE loaded successfully!")

# Step 2: Define a function to calculate the meaning preservation score
def calculate_meaning_preservation(input_sentence, transformed_sentence):
    """
    Calculate the semantic similarity between input and transformed sentences
    using the Universal Sentence Encoder (USE).
    
    Args:
    - input_sentence (str): Original sentence before transformation.
    - transformed_sentence (str): Sentence after transformation.
    
    Returns:
    - float: Meaning preservation score (similarity between -1 and 1).
    """
    # Create a list of sentences for embedding
    sentences = [input_sentence, transformed_sentence]
    
    # Compute embeddings for both sentences
    embeddings = use_model(sentences)
    
    # Convert embeddings to numpy arrays
    input_embedding, transformed_embedding = embeddings[0].numpy(), embeddings[1].numpy()
    
    # Compute the inner product (cosine similarity) between the two embeddings
    similarity_score = np.inner(input_embedding, transformed_embedding)
    
    return similarity_score


Loading Universal Sentence Encoder...
USE loaded successfully!


In [39]:
final_scores_model = []
for idx, (informal, formal) in enumerate(zip(X_test, generated_outputs)):
    score = calculate_meaning_preservation(informal, formal)
    final_scores_model.append(score)

In [40]:
final_scores_original = []
for idx, (informal, formal) in enumerate(zip(X_test, y_test)):
    score = calculate_meaning_preservation(informal, formal)
    final_scores_original.append(score)

In [43]:
meaning_preservation_model = np.array(final_scores_model).mean()
meaning_preservation_original = np.array(final_scores_original).mean()

In [48]:
meaning_preservation_score = (meaning_preservation_model)*100/meaning_preservation_original

In [49]:
print(meaning_preservation_score)

88.4582926214097


In [55]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Step 1: Load the Model and Tokenizer
model = BertForSequenceClassification.from_pretrained("/kaggle/working/model_configs_fscore")
tokenizer = BertTokenizer.from_pretrained("/kaggle/working/tokenizer_configs_fscore")

# Step 2: Tokenization
def formality_scores(input_text):
    text = input_text
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
    
    # Step 3: Prepare Input Tensors
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    
    # Move tensors to the appropriate device (CPU or GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    model.to(device)
    
    # Step 4: Forward Pass
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
    
    # Apply softmax to obtain probabilities
    probabilities = torch.softmax(logits, dim=1)
    
    # Extract formality scores
    formality_score = probabilities[0][1].item()  # Probability of class 1 (informal)

    return formality_score

In [69]:
formality_score_original = []
for i in range(len(X_test)):
    score = formality_scores(y_test[i])
    formality_score_original.append(score)

In [60]:
formality_score_model = []
for i in range(len(generated_outputs)):
    score = formality_scores(generated_outputs[i])
    formality_score_model.append(score)

In [70]:
formality_human = np.array(formality_score_original).mean()
formality_model = np.array(formality_score_model).mean()

In [72]:
formality_model/formality_human

0.6845521432266866